### Solución **Lanchain**

In [2]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Indexar

In [4]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

llm = ChatOpenAI(model='gpt-4o-mini')
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = InMemoryVectorStore(embeddings)

In [21]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  # Usa el vector store FAISS como ejemplo

# Función para obtener el loader correspondiente según la extensión del archivo
def get_loader(file_extension):
    if file_extension == 'txt':
        return TextLoader
    elif file_extension == 'pdf':
        return PyPDFLoader
    elif file_extension == 'csv':
        return CSVLoader
    else:
        raise ValueError(f"Formato no soportado: {file_extension}")

# Definir directorio donde se encuentran los archivos
DIRECTORY_PATH = './data'

# Extensiones de archivo y sus loaders correspondientes
SUPPORTED_FILE_TYPES = [
    (".txt", TextLoader),
    (".pdf", PyPDFLoader),
    (".csv", CSVLoader)
]

# Crear una lista para almacenar los loaders configurados
loaders = []

# Configurar loaders para cada tipo de archivo soportado
for extension, loader_class in SUPPORTED_FILE_TYPES:
    loader = DirectoryLoader(
        path=DIRECTORY_PATH,
        glob=f"**/*{extension}",
        loader_cls=loader_class
    )
    loaders.append(loader)

# Lista para almacenar los documentos cargados
documents = []

# Cargar documentos usando cada loader
for loader in loaders:
    loaded_documents = loader.load()
    documents.extend(loaded_documents)

# Imprimir información básica de los documentos cargados
print(f"Se cargaron {len(documents)} documentos.")

# Dividir documentos en fragmentos con RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño de cada fragmento (en caracteres)
    chunk_overlap=200,  # Superposición entre fragmentos (en caracteres)
    add_start_index=True  # Agregar el índice de inicio en el documento original
)

# Dividir los documentos en fragmentos más pequeños
all_splits = text_splitter.split_documents(documents)

print(f"Los documentos se dividieron en {len(all_splits)} fragmentos.")

# Agregar los fragmentos al vector store
document_ids = vector_store.add_documents(documents=all_splits)

# Imprimir los primeros tres IDs de documentos añadidos al vector store
print("Primeros 3 document IDs añadidos al vector store:", document_ids[:3])


Se cargaron 175 documentos.
Los documentos se dividieron en 769 fragmentos.
Primeros 3 document IDs añadidos al vector store: ['e3fe9041-acf5-4868-a3a7-e4c9ae9ff197', 'd7f3ed82-3b8e-4ec7-b8e4-9d69604e0a29', 'a31c5a3a-7a8c-4e88-bfba-0e6cbe1c5b49']


#### 2. Retrieval and Generation

In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:
